In [30]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sb

In [31]:
df = pd.read_csv('data.csv')
df.head()

,cum,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_ly,gdcd
0,1,1000001,3.6,5.00,4.0,NaN,NaN,NaN,2.75,6.0,8.75
1,1,1000002,8.4,6.75,7.6,NaN,NaN,NaN,8.50,7.5,8.25
2,1,1000003,5.8,7.50,5.0,NaN,NaN,NaN,7.25,5.5,8.75
3,1,1000004,7.4,7.50,8.6,NaN,NaN,NaN,7.50,6.5,7.50
4,1,1000005,7.2,8.50,9.0,NaN,NaN,NaN,8.00,8.5,8.25


### 1. Kiểm tra kiểu dữ liệu

In [32]:
print(df.dtypes)

cum            int64
sbd            int64
toan         float64
ngu_van      float64
ngoai_ngu    float64
vat_li       float64
hoa_hoc      float64
sinh_hoc     float64
lich_su      float64
dia_ly       float64
gdcd         float64
dtype: object


### 2. Toán, Văn, Ngoại ngữ - Thêm dữ liệu đối với NaN

In [33]:
# df[['toan', 'ngu_van']] = df[['toan', 'ngu_van']].fillna(df[['toan', 'ngu_van']].median())

- Toán và Ngữ Văn là hai môn bắt buộc phải thi, do đó nếu có giá trị nào là NaN thì gán nó bằng median

In [34]:
# df[['ngoai_ngu']] = df[['ngoai_ngu']].fillna(lambda x: 10 if x['toan'] >= 8 or x['ngu_van'] >= 8 else -1)

In [35]:
df.head()
df.count()

cum          995441
sbd          995441
toan         982726
ngu_van      981407
ngoai_ngu    870609
vat_li       325523
hoa_hoc      327367
sinh_hoc     322198
lich_su      659662
dia_ly       657421
gdcd         554343
dtype: int64

### 3. Tạo cột mới: khoi ('khtn' or 'khxh')

In [36]:
science = 'science'
khxh = 'khxh'
retest = 'retest'
# point vat_li >= 0  or hoa_hoc >= 0 or sinh_hoc >= 0 => science
# point dia_li >= 0  or su >= 0 or dia_lich >= 0 = > khxh
# Else => NaN

df['khoi'] = df.apply(lambda row: science if (row['vat_li'] >= 0 or row['hoa_hoc'] >= 0 or row['sinh_hoc'] >= 0) else khxh if (row['dia_ly'] >= 0 or row['lich_su'] >= 0 or row['gdcd'] >= 0) else np.nan, axis=1)

In [37]:
df[0:100]

,cum,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_ly,gdcd,khoi
0,1,1000001,3.6,5.00,4.0,NaN,NaN,NaN,2.75,6.00,8.75,khxh
1,1,1000002,8.4,6.75,7.6,NaN,NaN,NaN,8.50,7.50,8.25,khxh
2,1,1000003,5.8,7.50,5.0,NaN,NaN,NaN,7.25,5.50,8.75,khxh
3,1,1000004,7.4,7.50,8.6,NaN,NaN,NaN,7.50,6.50,7.50,khxh
4,1,1000005,7.2,8.50,9.0,NaN,NaN,NaN,8.00,8.50,8.25,khxh
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,1000099,7.6,8.00,9.2,NaN,NaN,NaN,5.00,8.50,8.00,khxh
96,1,1000100,3.8,5.50,5.6,NaN,NaN,NaN,5.50,4.25,7.75,khxh
97,1,1000101,8.2,6.50,7.6,5.25,7.25,3.25,NaN,NaN,NaN,khtn
98,1,1000102,8.8,7.25,9.0,4.75,8.00,8.00,NaN,NaN,NaN,khtn


### 4. Tách dữ liệu thành 2 phần: KHTN và KHXH

In [38]:
# KHTN
pf_khtn = df[df['khoi'] == khtn]
pf_khtn = pf_khtn.drop(['lich_su', 'gdcd', 'dia_ly'], axis=1)
# pf_khtn = pf_khtn.fillna(0)


# KHXH
pf_khxh = df[df['khoi'] == khxh]
pf_khxh = pf_khxh.drop(['vat_li', 'hoa_hoc', 'sinh_hoc'], axis=1)

In [39]:
pf_khtn.head()
# count row
pf_khtn.count()

cum          329235
sbd          329235
toan         329205
ngu_van      315709
ngoai_ngu    312709
vat_li       325523
hoa_hoc      327367
sinh_hoc     322198
khoi         329235
dtype: int64

In [40]:
pf_khxh.head()

,cum,sbd,toan,ngu_van,ngoai_ngu,lich_su,dia_ly,gdcd,khoi
0,1,1000001,3.6,5.00,4.0,2.75,6.0,8.75,khxh
1,1,1000002,8.4,6.75,7.6,8.50,7.5,8.25,khxh
2,1,1000003,5.8,7.50,5.0,7.25,5.5,8.75,khxh
3,1,1000004,7.4,7.50,8.6,7.50,6.5,7.50,khxh
4,1,1000005,7.2,8.50,9.0,8.00,8.5,8.25,khxh
